# Setup

In [1]:
# !pip install -U -q segmentation-models
# !pip install -q tensorflow==2.1
# !pip install -q keras==2.3.1
# !pip install -q tensorflow-estimator==2.1.

# ## Imports libs
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ["SM_FRAMEWORK"] = "tf.keras"

# from tensorflow import keras
# import segmentation_models as sm

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import Counter
from keras.preprocessing import sequence
import random

Using TensorFlow backend.


In [3]:
def create_letter_mapping(all_words):
    letter_li = []
    
    # Get unique letters in the data
    for word in all_words:
        for letter in word:
            letter_li.append(letter)
    
    # order letters by most common
    most_common = Counter(letter_li).most_common()
    letter_li = []
    for elem in most_common:
        letter_li.append(elem[0])
    
    # Map each letter to an integer
    letter_dict = {}
    for idx, letter in enumerate(letter_li):
        if letter_dict != '':
            letter_dict[letter] = idx + 1
        else:
            letter_dict[letter] = 0
    
    return letter_dict

def letter_to_number(word_li, letter_dict):
    return_li = []
    for letter in word_li:
        return_li.append(letter_dict[letter])
        
    return return_li

def pad_list(letter_li, pad_num=25):
    curr_len = len(letter_li)
    new_len = 25 - curr_len
    zeroes = [0] * new_len
    zeroes.extend(letter_li)
    return zeroes

# Fixing Language Groups

In [194]:
lan_df = pd.read_csv('language_data.csv')

In [195]:
lan_df.fillna('Other', inplace=True)

In [247]:
lan_df.loc[lan_df['full_name'].str.contains('English'), 'Group'] = 'Germanic'
lan_df.loc[lan_df['full_name'].str.contains('Dutch'), 'Group'] = 'Germanic'
lan_df.loc[lan_df['full_name'].str.contains('Norse'), 'Group'] = 'Germanic'
lan_df.loc[lan_df['full_name'].str.contains('Germ'), 'Group'] = 'Germanic'

lan_df.loc[lan_df['full_name'].str.contains('French'), 'Group'] = 'Latin'
lan_df.loc[lan_df['full_name'].str.contains('Greek'), 'Group'] = 'Greek'
lan_df.loc[lan_df['full_name'].str.contains('Hebrew'), 'Group'] = 'Other'

lan_df.loc[lan_df['full_name'] == 'Sanskrit', 'Group'] = 'Indo-Aryan'
lan_df.loc[lan_df['full_name'] == 'Hindi', 'Group'] = 'Indo-Aryan'

lan_df = lan_df[lan_df['Group'] != 'Arabic']
lan_df = lan_df[lan_df['Group'] != 'Turkish']
lan_df = lan_df[lan_df['Group'] != 'Indo-Aryan']


lan_df = lan_df[lan_df['Group'] != 'Other']

# Date Cleanup

In [248]:
lan_df.reset_index(inplace=True,drop=True)

In [249]:
group_dict = {}
for idx, value in enumerate(lan_df.Group.unique()):
    group_dict[value] = idx
    
lan_df['group_int'] = lan_df['Group'].apply(lambda x: group_dict[x])

In [250]:
group_dict

{'Germanic': 0, 'Greek': 1, 'Latin': 2, 'Japanese': 3}

In [251]:
lan_df['letter_li'] = lan_df['word'].apply(lambda x: list(x))

In [252]:
map_dict = create_letter_mapping(lan_df['word'].values)

In [253]:
lan_df['letter_ints'] = lan_df['letter_li'].apply(lambda x: pad_list(letter_to_number(x, map_dict)))

In [254]:
SHAPE = lan_df.shape[0]

In [255]:
word_arr = np.empty(SHAPE, dtype=list)
origin_arr = np.empty(SHAPE, dtype=int)

In [256]:
for i in range(SHAPE):
    word_arr[i] = lan_df.loc[i, 'letter_ints']
    origin_arr[i] = lan_df.loc[i, 'group_int']

In [257]:
# pad array so that all words have length of 25
word_arr = sequence.pad_sequences(word_arr, 25)

In [258]:
# Develop random index for train test split
random.seed(0)
rng = np.arange(SHAPE)
random.shuffle(rng)

In [259]:
train_len = int(SHAPE * 0.8)
train_idx = rng[:train_len]
test_idx = rng[train_len:]

In [260]:
X_train, y_train = word_arr[train_idx], origin_arr[train_idx]
X_test, y_test = word_arr[test_idx], origin_arr[test_idx]

# Models

In [335]:
embedding_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(map_dict) + 1, 64),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(len(group_dict), activation="softmax")
])

In [336]:
# classes = len(group_dict)
# embed_dims = 32
# stack_model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(len(map_dict) + 1, embed_dims),
#     tf.keras.layers.LSTM(64, return_sequences=True),
#     tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.Dense(classes, activation="softmax")
# ])

In [ ]:
embedding_model.compile(loss="sparse_categorical_crossentropy",optimizer="rmsprop",metrics=['acc'])
history = embedding_model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Train on 35738 samples, validate on 8935 samples
Epoch 1/10
35738/35738 [==============================] - 26s 738us/sample - loss: 0.7580 - acc: 0.6633 - val_loss: 0.6923 - val_acc: 0.6891
Epoch 2/10
35738/35738 [==============================] - 22s 629us/sample - loss: 0.6744 - acc: 0.7039 - val_loss: 0.6587 - val_acc: 0.7080
Epoch 3/10
35738/35738 [==============================] - 23s 657us/sample - loss: 0.6361 - acc: 0.7224 - val_loss: 0.6260 - val_acc: 0.7246
Epoch 4/10
35738/35738 [==============================] - 23s 634us/sample - loss: 0.6027 - acc: 0.7397 - val_loss: 0.5887 - val_acc: 0.7438
Epoch 5/10
35738/35738 [==============================] - 24s 663us/sample - loss: 0.5755 - acc: 0.7525 - val_loss: 0.5748 - val_acc: 0.7548
Epoch 6/10
35738/35738 [==============================] - 24s 665us/sample - loss: 0.5551 - acc: 0.7655 - val_loss: 0.5549 - val_acc: 0.7682
Epoch 7/10
35738/35738 [==============================] - 24s 682us/sample - loss: 0.5325 - acc: 0.7764 -

In [295]:
# stack_model.compile(loss="sparse_categorical_crossentropy",optimizer="rmsprop",metrics=['acc'])
# history_1 = embedding_model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Predicting Individual Words

In [323]:
def get_word_prediction(test_word, model):
    temp = pad_list(letter_to_number(list(test_word), map_dict))
    temp = model.predict(np.array([temp]))
    temp_df = pd.DataFrame(data={'group':[elem for elem in group_dict.keys()], 'probability' : temp[0] * 100})
    return temp_df

In [331]:
get_word_prediction('geometry', embedding_model)

,group,probability
0,Germanic,37.569405
1,Greek,13.279668
2,Latin,49.131413
3,Japanese,0.019517
